In [1]:
import os
import sys
sys.path.append('../')
os.chdir('../')

In [2]:
import pandas as pd
import duck_db_helper
import duckdb

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
conn = duckdb.connect('../data/analytical_sandboxes/analytical_sandbox_zone.db')
tables = duck_db_helper.get_tables(conn)
df_list = []
df_list_names = []
for table_name in tables: 
    df = conn.sql(f"SELECT * FROM \"{table_name}\";").df()
    df_list.append(df)
    df_list_names.append(table_name)
conn.close()
dfm = df_list[0]
dfp = df_list[1]

# Joining the two tables

We will add for each player record of a match, the match statistics we have from the matches tables. This will add new features like the betting odds and the weather. 


In [4]:
df_home = pd.merge(dfp, dfm, left_on=['team_x', 'match_date'], right_on=['HomeTeam', 'Date'], how='inner')
df_away = pd.merge(dfp, dfm, left_on=['team_x', 'match_date'], right_on=['AwayTeam', 'Date'], how='inner')
df = pd.concat([df_home, df_away], ignore_index=True)
df

,match_date,team_x,opp_team_name,was_home,name,position,assists,bonus,bps,clean_sheets,creativity,element,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,yellow_cards,red_cards,round,saves,selected,threat,total_points,value,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AvgH,AvgD,AvgA,PRESS,WDIR,WSPD,CLOUD,TEMP,TDEW
0,2022-08-14,Nott'm Forest,West Ham,1,Brennan Johnson,FWD,0,0,5,1,14.3,394,0,0,3.3,1.6,90,0,0,0,1,0,2,0,160914,17,1,60,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
1,2022-08-14,Nott'm Forest,West Ham,1,Orel Mangala,MID,0,0,4,1,4.4,536,0,0,1.4,1.6,83,0,0,0,0,0,2,0,3453,8,3,50,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
2,2022-08-14,Nott'm Forest,West Ham,1,Dean Henderson,GK,0,3,43,1,0.0,398,0,0,5.0,50.0,90,0,0,1,1,0,2,4,466426,0,14,45,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
3,2022-08-14,Nott'm Forest,West Ham,1,Taiwo Awoniyi,FWD,0,2,26,1,7.2,397,0,1,5.6,31.6,71,0,0,0,0,0,2,0,78654,17,8,59,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
4,2022-08-14,Nott'm Forest,West Ham,1,Harry Toffolo,DEF,0,1,24,1,26.8,523,0,0,4.7,20.6,90,0,0,0,0,0,2,0,8329,0,7,45,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7300,2023-05-28,Wolves,Arsenal,0,Matheus Luiz Nunes,MID,0,0,3,0,0.0,589,3,0,0.0,0.0,45,0,0,0,0,0,38,0,17059,0,1,48,2023-05-28,Arsenal,Wolves,5,0,H,1.37,5.33,7.87,1023.4,NE,3.0,0.0,12.100000,8.100000
7301,2023-05-28,Wolves,Arsenal,0,Rúben da Silva Neves,MID,0,0,4,0,0.0,480,2,0,0.0,0.0,45,0,0,0,0,0,38,0,175772,0,1,52,2023-05-28,Arsenal,Wolves,5,0,H,1.37,5.33,7.87,1023.4,NE,3.0,0.0,12.100000,8.100000
7302,2023-05-28,Wolves,Arsenal,0,Nélson Cabral Semedo,DEF,0,0,15,0,0.0,482,5,0,0.0,0.0,90,0,0,0,0,0,38,0,73746,0,0,50,2023-05-28,Arsenal,Wolves,5,0,H,1.37,5.33,7.87,1023.4,NE,3.0,0.0,12.100000,8.100000
7303,2023-05-28,Wolves,Arsenal,0,Nathan Collins,DEF,0,0,14,0,0.0,516,5,0,0.0,0.0,90,0,0,0,0,0,38,0,57150,0,0,42,2023-05-28,Arsenal,Wolves,5,0,H,1.37,5.33,7.87,1023.4,NE,3.0,0.0,12.100000,8.100000


Now we will get rid of some of the columns used for the merge that are no longer needed

In [5]:
cols_to_remove = ['Date','HomeTeam','AwayTeam']
cols_to_remove = [col for col in cols_to_remove if col in df.columns]
df = df.drop(cols_to_remove,axis=1)

# Generating odds for home or not

We do simple feature generation to get the odds for the players team and the odds agaings the players team by looking at if the player was at home or not.
TODO if this is the only feature generation we are going to do, i suggest we remove all the game from the analytical sandbox. They just clutter up the tables and we never use them.

In [6]:
home_odds_col = 'AvgH'
draw_odds_col = 'AvgD'
away_odds_col = 'AvgA'

df['odds_against'] = 0.0
df['odds_for'] = 0.0

# Fill in odds columns based on whether the player was home or not
df.loc[df['was_home'] == 1, 'odds_for'] = df.loc[df['was_home'] == 1, home_odds_col]
df.loc[df['was_home'] == 0, 'odds_for'] = df.loc[df['was_home'] == 0, away_odds_col]
df.loc[df['was_home'] == 1, 'odds_against'] = df.loc[df['was_home'] == 1, away_odds_col]
df.loc[df['was_home'] == 0, 'odds_against'] = df.loc[df['was_home'] == 0, home_odds_col]

df.drop(columns=[home_odds_col, away_odds_col], inplace=True)
df.rename(columns={'AvgD': 'odds_draw'}, inplace=True)

In [7]:
df.loc[df.was_home==0].head(5)

,match_date,team_x,opp_team_name,was_home,name,position,assists,bonus,bps,clean_sheets,creativity,element,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,yellow_cards,red_cards,round,saves,selected,threat,total_points,value,FTHG,FTAG,FTR,odds_draw,PRESS,WDIR,WSPD,CLOUD,TEMP,TDEW,odds_against,odds_for
3658,2022-08-13,Newcastle,Brighton,0,Fabian Schär,DEF,0,0,22,1,0.7,366,0,0,2.1,20.6,90,0,0,0,1,0,2,0,411488,0,5,46,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1
3659,2022-08-13,Newcastle,Brighton,0,Joe Willock,MID,0,0,11,1,5.2,373,0,0,5.2,20.2,75,0,0,0,0,0,2,0,55473,27,3,50,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1
3660,2022-08-13,Newcastle,Brighton,0,Kieran Trippier,DEF,0,1,28,1,15.9,357,0,0,3.7,20.6,90,0,0,0,0,0,2,0,1867358,0,7,50,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1
3661,2022-08-13,Newcastle,Brighton,0,Joelinton Cássio Apolinário de Lira,MID,0,0,10,1,14.0,371,0,0,3.0,14.2,90,0,0,0,1,0,2,0,77813,2,2,60,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1
3662,2022-08-13,Newcastle,Brighton,0,Sven Botman,DEF,0,1,28,1,0.3,377,0,0,1.2,11.8,90,0,0,0,0,0,2,0,186217,0,7,45,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1


In [9]:
conn = duckdb.connect('../data/feature_generation/feature_generation.db')
duck_db_helper.create_table('feature_1', df, conn)
conn.close()